In [1]:
# default_exp codexplainer.metrics_java

In [19]:
# export

import pandas as pd
import numpy as np

from ds4se.exp.metrics_java import *
from ds4se.mgmnt.prep.files_mgmnt import *
from ds4se.mgmnt.prep.token_mgmnt import *
from ds4se.codexplainer.utils import *

from ds4se.utils.distances import statistical_distance
from ds4se.utils.statistics import bootstrapping

from tqdm import tqdm

from typing import Optional, List, Tuple, Callable, Dict

In [2]:
# export

#Logging configuration

import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.INFO)

### metrics explainer

> This module provides methods for performing comparisons between code samples leveraging metrics module (java) 

> @Alvaro

## Parameterization

In [3]:
# export

def _get_metrics_cols(columns: List[str]):
    cols = [col for col in columns if (col != "file" and col != "class")]
    return cols

In [ ]:
# export

def _gather_description_stats(metrics: List[str], exp_stats_dict: Dict, stats: List[str],
             decript_df: pd.DataFrame):
    """
    Gather data related to a set of errors and a set of statistics behavior (e.g., mean, median)
    
    """
    for met in metrics:
        if met not in exp_stats_dict:
            exp_stats_dict[met] = {}
            for stat in stats:
                exp_stats_dict[met][stat] = {}
                
        for stat in stats:
            exp_stats_dict[met][stat] = decript_df[met].loc[stat]

def _gather_stats_for_experiment(gen_metrics: pd.DataFrame, hmn_metrics: pd.DataFrame,
                                stats: List[str]) -> Tuple[Dict, Dict]:
    """
    Gather the data for 2 sets of errors, regarding the behavior of several statistics (e.g, mean)
    for a single experiment
    
    :param gen_errors: pd.DF containing errors data for generator model
    :param hmn_errors: pd.DF containing errors data for human data
    :param stats: List containing the stats of interest
    
    :return: Tuple[Dict, Dict] containing the data for both error samples (model, human)
    """
    gen_metrics_cols = _get_metrics_cols(list(gen_metrics.columns))
    
    gen_metrics_descript = gen_metrics.describe()
    hmn_metrics_descript = hmn_metrics.describe()
    
    gen_exp_stats = {}
    hmn_exp_stats = {}
    
    _gather_description_stats(gen_metrics_cols, gen_exp_stats, stats, gen_metrics_descript)
    _gather_description_stats(gen_metrics_cols, hmn_exp_stats, stats, hmn_metrics_descript)
    
    return gen_exp_stats, hmn_exp_stats

In [ ]:
# export

def verify_if_zeros(array: np.ndarray):
    return np.all((np.array(array) == 0))

In [ ]:
# export

def compare_metrics(gen_metrics: pd.DataFrame, hmn_metrics: pd.DataFrame,
                    est_func: Callable, bs_size: int, bins_method: Optional[str]='auto'):
    """
    Perform dimension-wise comparison of given metrics sets
    """
    div_dist_data = {}
    
    gen_dims = _get_metrics_cols(list(gen_metrics.columns))
    
    for dimension in gen_dims:
        gen_dim_data = gen_metrics[dimension].values
        hmn_dim_data = hmn_metrics[dimension].values
        
        gen_dim_data_zeros = verify_if_zeros(gen_dim_data)
        hmn_dim_data_zeros = verify_if_zeros(hmn_dim_data)
        
        # Bootstrapping should not be performed here
        # but when gathering data for estimating experiments' behavior instead
        
#         logging.info("Starting bootstrapping.")
#         bs_gen_dim_data = bootstrapping(gen_dim_data, est_func, sample_size=bs_size, full_zeros=gen_dim_data_zeros)
#         bs_hmn_dim_data = bootstrapping(hmn_dim_data, est_func, sample_size=bs_size, full_zeros=hmn_dim_data_zeros)
        
#         bs_gen_dim = bs_gen_dim_data['bootstrap_repl'].values
#         bs_hmn_dim = bs_hmn_dim_data['bootstrap_repl'].values
            
        
        logging.info("Starting JS computations.")
        # Bins setting is estimated via Freedman–Diaconis rule
        js_div, js_dist = statistical_distance(hmn_dim_data, gen_dim_data)
        
        div_dist_data[dimension] = {
            "JS-Divergence": js_div,
            "JS-Distance": js_dist
        }
        
    return div_dist_data

In [ ]:
# export

def _group_js_exp_data_by_metrics_categories(experiment_list: List,
                                             categories: Optional[Dict]=metrics_categories
                                             )-> Tuple[Dict, Dict]:
    """
    Group data according to established metrics categories
    :param experiment_list: List with dictionaries containing data for categories
    :param categories:
    """
    
    # Init structure for gathering data
    # -------------
    grouped_data_dist = {}
    grouped_data_div = {}

    for cat in categories:
        grouped_data_dist[cat] = []
        grouped_data_div[cat] = []
    # -------------
    
    # Gather data by categories
    for experiment in experiment_list:
        for cat in categories:
            cat_dims = categories[cat]
            logging.info(f"Metrics dimensions for category: {cat}")
            
            exp_div_data = {}
            exp_dist_data = {}
            
            for dim, measures in experiment.items():
                if dim not in cat_dims:
                    continue

                exp_div_data[dim] = measures['JS-Divergence']
                exp_dist_data[dim] = measures['JS-Distance']
        
            grouped_data_dist[cat].append(exp_dist_data)
            grouped_data_div[cat].append(exp_div_data)
            
    return grouped_data_dist, grouped_data_div

In [ ]:
# export 

def perform_metrics_distance_analysis(gen_samples_path: str, hmn_samples_path: str,
                                      ck_tool_jar: str, n_experiments: int,
                                      physical_files_path: str,
                                      out_path: str,
                                      rm_java_files: Optional[bool]=True,
                                      samples_code_col: Optional[str]="code",
                                      np_est_function: Optional[Callable]=np.mean,
                                      bootstrap_samples_size: Optional[int]=500,
                                      samples_extension: Optional[str]="jsonl",
                                      compar_stats: Optional[List]=['mean'],
                                      group_by_metrics_categories: Optional[bool]=True,
                                      process_tokens: Optional[bool]=True,
                                      bins_method: Optional[str]='auto'
                                     ):
    """
    Perform the analysis for getting comparison info. about
    2 sets of samples (generator vs human)
    
    
    :param gen_samples_path: Str indicating path containing generator samples
    :param hmn_samples_path: Str indicating path containing human samples
    :param ck_tool_jar: Str indicating path containing aux. CK tool (for computing metrics)
    :param n_experiments: int indicating number of experiments to consider
    :param physical_files_path: Str indicating location for (tmp) java files
    :param rm_java_files: bool indicating whether to remove generated java files or not
    :param samples_code_col: Str indicating column name containing code in sample sets
    :param np_est_function: np function indicating indicating estimator for boostraping
                            - Default: np.mean    
    :param bootstrap_samples_size: int indicating the size for resample sets used
                                   in bootstrapping
    :param samples_extension: Str indicating file extension for sample files
                              - Default: jsonl
    :param compar_stats: Statistics of interest for gathering data about
                         - Default: ['mean']
    
    :return: Tuple[DataFrame, DataFrame, Dict, Dict, Dict, Dict]
             Containing:
             - Distance data for all experiments
             - Divergence data for all experiments
             - Generator samples metrics stats (according to specified input) accross all experiments
             - Human samples metrics stats (according to specified input) accross all experiments
             - Metric-based grouping for distance data
             - Metric-based grouping for divergence data             
    """
    
    # Data structures for gathering data across experiments
    distance_data = []
    gen_metrics_stats = []
    hmn_metrics_stats = []
    categorized_js_data = []
    
    logging.info("Building metrics analyzer.")
    
    java_analyzer = JavaAnalyzer(ck_tool_jar)
    
    # Get sample file names
    gen_sample_set = get_files_list(gen_samples_path, samples_extension)
    hmn_sample_set = get_files_list(hmn_samples_path, samples_extension)
    
    for i in tqdm(range(n_experiments)):
        gen_sample_path = gen_sample_set[i]
        hmn_sample_path = hmn_sample_set[i]
        
        gen_samples = jsonl_to_dataframe(str(gen_sample_path))
        hmn_samples = jsonl_to_dataframe(str(hmn_sample_path))
        
        if process_tokens:
            gen_samples = replace_spec_toks_to_original(gen_samples, java_special_tokens)
            hmn_samples = replace_spec_toks_to_original(hmn_samples, java_special_tokens)
        
        logging.info('Gen samples: ')
        logging.info(gen_samples.head())
        
        logging.info('Human samples: ')
        logging.info(hmn_samples.head())
    
        logging.info("Gathering metrics for samples.")
        gen_metrics = java_analyzer.compute_metrics(gen_samples[samples_code_col], physical_files_path, remove_java_files=rm_java_files)
        hmn_metrics = java_analyzer.compute_metrics(hmn_samples[samples_code_col], physical_files_path, remove_java_files=rm_java_files)
        
        # ------------------------------------------------------
        # Save metrics
        gen_metrics.to_csv(f"{out_path}/{gen_sample_path.stem}-gen-metrics.csv")
        hmn_metrics.to_csv(f"{out_path}/{hmn_sample_path.stem}-hmn-metrics.csv")
        # ------------------------------------------------------
        
        logging.info("Performing comparison.")
        experiment_comp = compare_metrics(gen_metrics, hmn_metrics, np_est_function, bootstrap_samples_size, bins_method)
        distance_data.append(experiment_comp)
        
        logging.info(f"Gathering statistics for: {compar_stats}...")
        gen_exp_stats, hmn_exp_stats = _gather_stats_for_experiment(gen_metrics, hmn_metrics, compar_stats)
        gen_metrics_stats.append(gen_exp_stats)
        hmn_metrics_stats.append(hmn_exp_stats)
        
    grouped_distances, grouped_divs = {}, {}
    if group_by_metrics_categories:
        grouped_distances, grouped_divs= _group_js_exp_data_by_metrics_categories(distance_data)
        
        
    distance_df, divergence_df, gen_mtcs_stats_dict, hmn_mtcs_stats_dict = get_tabular_data(
        distance_data, gen_metrics_stats, hmn_metrics_stats, compar_stats
    )
    
    return distance_df, divergence_df, gen_mtcs_stats_dict, hmn_mtcs_stats_dict, grouped_distances, grouped_divs

In [ ]:
# export

def get_tabular_data(distance_data, gen_metrics_stats, hmn_metrics_stats, compar_stats):
    distance_df, divergence_df = get_tabulardata_distances_data(distance_data)
    gen_mtcs_stats_dict = get_data_stats_dataframe(gen_metrics_stats, compar_stats)
    hmn_mtcs_stats_dict = get_data_stats_dataframe(hmn_metrics_stats, compar_stats)
    
    return distance_df, divergence_df, gen_mtcs_stats_dict, hmn_mtcs_stats_dict

In [20]:
from nbdev.export import notebook2script
notebook2script()

Converted 0.1_mgmnt.prep.ipynb.
Converted 0.2_mgmnt.prep.files_mgmnt.ipynb.
Converted 0.3_mgmnt.prep.bpe_tokenization.ipynb.
Converted 0.4_mgmnt.prep.tokenization_counting.ipynb.
Converted 0.5_mgmnt.prep.token_mgmnt.ipynb.
Converted 1.1_exp.info.ipynb.
Converted 1.2_exp.desc.metrics.java.ipynb.
Converted 1.4_exp.metrics_python.ipynb.
Converted 1.5_exp.metrics_java.ipynb.
Converted 2.0_repr.codebert.ipynb.
Converted 2.0_repr.i.ipynb.
Converted 2.1_repr.codeberta.ipynb.
Converted 2.1_repr.roberta.train.ipynb.
Converted 2.2_repr.roberta.eval.ipynb.
Converted 2.3_repr.word2vec.train.ipynb.
Converted 2.6_repr.word2vec.eval.ipynb.
Converted 2.7_repr.distmetrics.ipynb.
Converted 2.8_repr.sentence_transformers.ipynb.
Converted 3.1_traceability.unsupervised.eda.ipynb.
Converted 3.2_traceability.unsupervised.approach.d2v.ipynb.
Converted 3.2_traceability.unsupervised.approach.w2v.ipynb.
Converted 4.0_infoxplainer.ir.ipynb.
Converted 4.1_infoxplainer.ir.unsupervised.d2v.ipynb.
Converted 4.2_infox